In [1]:

# TODO : Include the PYQGIS imports for the plugin
from pymeos.db.psycopg import MobilityDB
from pymeos import *
from datetime import datetime, timedelta
import time
from collections import deque
from pympler import asizeof
import gc
from enum import Enum
import numpy as np
from shapely.geometry import Point
import math
import subprocess

from datetime import datetime, timedelta



class Database_connector:
    """
    Singleton class used to connect to the MobilityDB database.
    """
    
    def __init__(self):
        try: 
            connection_params = {
            "host": "localhost",
            "port": 5432,
            "dbname": "mobilitydb",
            "user": "postgres",
            "password": "postgres"
            }
            self.table_name = "PyMEOS_demo"
            self.id_column_name = "MMSI"
            self.tpoint_column_name = "trajectory"                    
            self.connection = MobilityDB.connect(**connection_params)

            self.cursor = self.connection.cursor()

            self.cursor.execute(f"SELECT {self.id_column_name} FROM public.{self.table_name};")
            self.ids_list = self.cursor.fetchall()
            self.ids_list = self.ids_list[:int(len(self.ids_list)*PERCENTAGE_OF_OBJECTS)]
        except Exception as e:
            pass

  
    def get_subset_of_tpoints(self, pstart, pend, xmin, ymin, xmax, ymax):
        """
        For each object in the ids_list :
            Fetch the subset of the associated Tpoints between the start and end timestamps
            contained in the STBOX defined by the xmin, ymin, xmax, ymax.
        """
        try:
           
            ids_list = [ f"'{id[0]}'"  for id in self.ids_list]
            ids_str = ', '.join(map(str, ids_list))
          
            query = f"""
                    SELECT 
                        atStbox(
                            a.{self.tpoint_column_name}::tgeompoint,
                            stbox(
                                ST_MakeEnvelope(
                                    {xmin}, {ymin}, -- xmin, ymin
                                    {xmax}, {ymax}, -- xmax, ymax
                                    4326 -- SRID
                                ),
                                tstzspan('[{pstart}, {pend}]')
                            )
                        )
                    FROM public.{self.table_name} as a 
                    WHERE a.{self.id_column_name} in ({ids_str});
                    """
            self.cursor.execute(query)
            rows = self.cursor.fetchall()
            return rows
        except Exception as e:
            self.log(e)


    def get_min_timestamp(self):
        """
        Returns the min timestamp of the tpoints columns.

        """
        try:
            
            self.cursor.execute(f"SELECT MIN(startTimestamp({self.tpoint_column_name})) AS earliest_timestamp FROM public.{self.table_name};")
            return self.cursor.fetchone()[0]
        except Exception as e:
            pass

    def get_max_timestamp(self):
        """
        Returns the max timestamp of the tpoints columns.

        """
        try:
            self.cursor.execute(f"SELECT MAX(endTimestamp({self.tpoint_column_name})) AS latest_timestamp FROM public.{self.table_name};")
            return self.cursor.fetchone()[0]
        except Exception as e:
            pass


    def close(self):
        """
        Close the connection to the MobilityDB database.
        """
        self.cursor.close()
        self.connection.close()




FPS_DEQUEUE_SIZE = 5 # Length of the dequeue to calculate the average FPS
TIME_DELTA_DEQUEUE_SIZE =  10 # Length of the dequeue to keep the keys to keep in the buffer


PERCENTAGE_OF_OBJECTS = 1 # To not overload the memory, we only take a percentage of the ships in the database
TIME_DELTA_SIZE = 240 # Number of frames associated to one Time delta
GRANULARITY = timedelta(minutes=1) # Time delta between two frames
SRID = 4326
FPS = 60



  

In [2]:
db = Database_connector()

start_date = db.get_min_timestamp()
end_date = db.get_max_timestamp()
total_frames = math.ceil( (end_date - start_date) // GRANULARITY )

timestamps = [start_date + i * GRANULARITY for i in range(total_frames)]
timestamps = [dt.replace(tzinfo=None) for dt in timestamps]
timestamps_strings = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in timestamps]

In [3]:
timestamps

[datetime.datetime(2023, 6, 1, 0, 0),
 datetime.datetime(2023, 6, 1, 0, 1),
 datetime.datetime(2023, 6, 1, 0, 2),
 datetime.datetime(2023, 6, 1, 0, 3),
 datetime.datetime(2023, 6, 1, 0, 4),
 datetime.datetime(2023, 6, 1, 0, 5),
 datetime.datetime(2023, 6, 1, 0, 6),
 datetime.datetime(2023, 6, 1, 0, 7),
 datetime.datetime(2023, 6, 1, 0, 8),
 datetime.datetime(2023, 6, 1, 0, 9),
 datetime.datetime(2023, 6, 1, 0, 10),
 datetime.datetime(2023, 6, 1, 0, 11),
 datetime.datetime(2023, 6, 1, 0, 12),
 datetime.datetime(2023, 6, 1, 0, 13),
 datetime.datetime(2023, 6, 1, 0, 14),
 datetime.datetime(2023, 6, 1, 0, 15),
 datetime.datetime(2023, 6, 1, 0, 16),
 datetime.datetime(2023, 6, 1, 0, 17),
 datetime.datetime(2023, 6, 1, 0, 18),
 datetime.datetime(2023, 6, 1, 0, 19),
 datetime.datetime(2023, 6, 1, 0, 20),
 datetime.datetime(2023, 6, 1, 0, 21),
 datetime.datetime(2023, 6, 1, 0, 22),
 datetime.datetime(2023, 6, 1, 0, 23),
 datetime.datetime(2023, 6, 1, 0, 24),
 datetime.datetime(2023, 6, 1, 0, 2

In [18]:
x_min = -180
y_min = -90
x_max = 180
y_max = 90

start_frame= 0
end_frame = 240

pstart =   timestamps_strings[start_frame]
pend = timestamps_strings[end_frame]

arguments = [start_frame, end_frame, PERCENTAGE_OF_OBJECTS, x_min, y_min, x_max, y_max]
arguments = [str(arg) for arg in arguments]
arguments += [timestamps_strings[0], str(len(timestamps_strings)), "MINUTE", "/home/ali/matrices"]


In [19]:
arguments

['0',
 '240',
 '1',
 '-180',
 '-90',
 '180',
 '90',
 '2023-06-01 00:00:00',
 '1439',
 'MINUTE',
 '/home/ali/matrices']

In [20]:
# Command to execute Program B
command = ['/usr/bin/python3', '/home/ali/QGIS-MobilityDB/experiment8_subprocess/matrix_file.py', *arguments]
result = subprocess.run(command, capture_output=True, text=True)

In [21]:
result

CompletedProcess(args=['/usr/bin/python3', '/home/ali/QGIS-MobilityDB/experiment8_subprocess/matrix_file.py', '0', '240', '1', '-180', '-90', '180', '90', '2023-06-01 00:00:00', '1439', 'MINUTE', '/home/ali/matrices'], returncode=0, stdout='================================================================     Matrix 0 created in 10.678029298782349 seconds, 320.34087896347046 frames for 30 fps animation.\n', stderr='')

import numpy as np

# Load the .npy file with allow_pickle set to True
loaded_matrix = np.load('matrix_0.npy', allow_pickle=True)

print(loaded_matrix)


# Debugging process B code

In [7]:
sysargs = arguments
sysargs

['0',
 '240',
 '1',
 '-180',
 '-90',
 '180',
 '90',
 '2023-06-01 00:00:00',
 '2023-06-01 04:00:00',
 '2023-06-01 00:00:00',
 '2023-06-01 00:01:00',
 '2023-06-01 00:02:00',
 '2023-06-01 00:03:00',
 '2023-06-01 00:04:00',
 '2023-06-01 00:05:00',
 '2023-06-01 00:06:00',
 '2023-06-01 00:07:00',
 '2023-06-01 00:08:00',
 '2023-06-01 00:09:00',
 '2023-06-01 00:10:00',
 '2023-06-01 00:11:00',
 '2023-06-01 00:12:00',
 '2023-06-01 00:13:00',
 '2023-06-01 00:14:00',
 '2023-06-01 00:15:00',
 '2023-06-01 00:16:00',
 '2023-06-01 00:17:00',
 '2023-06-01 00:18:00',
 '2023-06-01 00:19:00',
 '2023-06-01 00:20:00',
 '2023-06-01 00:21:00',
 '2023-06-01 00:22:00',
 '2023-06-01 00:23:00',
 '2023-06-01 00:24:00',
 '2023-06-01 00:25:00',
 '2023-06-01 00:26:00',
 '2023-06-01 00:27:00',
 '2023-06-01 00:28:00',
 '2023-06-01 00:29:00',
 '2023-06-01 00:30:00',
 '2023-06-01 00:31:00',
 '2023-06-01 00:32:00',
 '2023-06-01 00:33:00',
 '2023-06-01 00:34:00',
 '2023-06-01 00:35:00',
 '2023-06-01 00:36:00',
 '2023-06-01

In [18]:
import numpy as np
from shapely.geometry import Point
import h5py
from pymeos.db.psycopg import MobilityDB

from pymeos import *
import os
import sys
from datetime import timedelta, datetime
from pymeos import *
import time

now = time.time()

FPS_DEQUEUE_SIZE = 5 # Length of the dequeue to calculate the average FPS
TIME_DELTA_DEQUEUE_SIZE =  10 # Length of the dequeue to keep the keys to keep in the buffer


args = sysargs

begin_frame = 240
end_frame = 479
TIME_DELTA_SIZE = end_frame - begin_frame + 1
PERCENTAGE_OF_OBJECTS = float(args[2])


GRANULARITY = timedelta(minutes=1) # Time delta between two frames
SRID = 4326
FPS = 60



class Database_connector:
    """
    Singleton class used to connect to the MobilityDB database.
    """
    
    def __init__(self):
        try: 
            connection_params = {
            "host": "localhost",
            "port": 5432,
            "dbname": "mobilitydb",
            "user": "postgres",
            "password": "postgres"
            }
            self.table_name = "PyMEOS_demo"
            self.id_column_name = "MMSI"
            self.tpoint_column_name = "trajectory"                    
            self.connection = MobilityDB.connect(**connection_params)

            self.cursor = self.connection.cursor()

            self.cursor.execute(f"SELECT {self.id_column_name} FROM public.{self.table_name};")
            self.ids_list = self.cursor.fetchall()
            self.ids_list = self.ids_list[:int(len(self.ids_list)*PERCENTAGE_OF_OBJECTS)]
        except Exception as e:
            pass

  
    def get_subset_of_tpoints(self, pstart, pend, xmin, ymin, xmax, ymax):
        """
        For each object in the ids_list :
            Fetch the subset of the associated Tpoints between the start and end timestamps
            contained in the STBOX defined by the xmin, ymin, xmax, ymax.
        """
        try:
           
            ids_list = [ f"'{id[0]}'"  for id in self.ids_list]
            ids_str = ', '.join(map(str, ids_list))
          
            query = f"""
                    SELECT 
                        atStbox(
                            a.{self.tpoint_column_name}::tgeompoint,
                            stbox(
                                ST_MakeEnvelope(
                                    {xmin}, {ymin}, -- xmin, ymin
                                    {xmax}, {ymax}, -- xmax, ymax
                                    4326 -- SRID
                                ),
                                tstzspan('[{pstart}, {pend}]')
                            )
                        )
                    FROM public.{self.table_name} as a 
                    WHERE a.{self.id_column_name} in ({ids_str});
                    """
            self.cursor.execute(query)
            # print(query)
            rows = self.cursor.fetchall()
            return rows
        except Exception as e:
            # print(e)
            pass


    def get_min_timestamp(self):
        """
        Returns the min timestamp of the tpoints columns.

        """
        try:
            
            self.cursor.execute(f"SELECT MIN(startTimestamp({self.tpoint_column_name})) AS earliest_timestamp FROM public.{self.table_name};")
            return self.cursor.fetchone()[0]
        except Exception as e:
            pass

    def get_max_timestamp(self):
        """
        Returns the max timestamp of the tpoints columns.

        """
        try:
            self.cursor.execute(f"SELECT MAX(endTimestamp({self.tpoint_column_name})) AS latest_timestamp FROM public.{self.table_name};")
            return self.cursor.fetchone()[0]
        except Exception as e:
            pass


    def close(self):
        """
        Close the connection to the MobilityDB database.
        """
        self.cursor.close()
        self.connection.close()



file_name = f"/home/ali/matrices/matrix_{begin_frame}.npy"
GRANULARITY = timedelta(minutes=1)

# check if file does't already exist


pymeos_initialize()
db = Database_connector()

x_min = float(args[3])
y_min = float(args[4])
x_max = float(args[5])
y_max = float(args[6])

timestamps = args[7:]
# transform the timestamps to datetime objects
timestamps = [datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S') for timestamp in timestamps]

p_start = timestamps[begin_frame]
p_end = timestamps[end_frame]
# print(p_start, p_end, x_min, y_min, x_max, y_max)
rows = db.get_subset_of_tpoints(p_start, p_end, x_min, y_min, x_max, y_max)    
    
        
empty_point_wkt = Point().wkt  # "POINT EMPTY"
matrix = np.full((len(rows), TIME_DELTA_SIZE), empty_point_wkt, dtype=object)

time_ranges = timestamps
now = time.time()


for i in range(len(rows)):
    try:
        traj = rows[i][0]
        traj = traj.temporal_precision(GRANULARITY) 
        num_instants = traj.num_instants()
        if num_instants == 0:
            continue
        elif num_instants == 1:
            single_timestamp = traj.timestamps()[0].replace(tzinfo=None)
            index = time_ranges.index(single_timestamp) - begin_frame
            matrix[i][index] = traj.values()[0].wkt
            
        elif num_instants >= 2:
            traj_resampled = traj.temporal_sample(start=time_ranges[0],duration= GRANULARITY)
            
            start_index = time_ranges.index( traj_resampled.start_timestamp().replace(tzinfo=None) ) - begin_frame
            end_index = time_ranges.index( traj_resampled.end_timestamp().replace(tzinfo=None) ) - begin_frame
    
            trajectory_array = np.array([point.wkt for point in traj_resampled.values()])
            matrix[i, start_index:end_index+1] = trajectory_array
    
    except:
        continue

# np.save(file_name, matrix)

db.close()
pymeos_finalize()
total_time = time.time() - now
frames_for_30_fps= 30 * total_time
print(f"================================================================     Matrix {begin_frame} created in {total_time} seconds, {frames_for_30_fps} frames for 30 fps animation.")

================================================================     Matrix 240 created in 12.534109115600586 seconds, 376.0232734680176 frames for 30 fps animation.


In [19]:
len(rows)

5821

In [20]:
np.count_nonzero(matrix != 'POINT EMPTY')

864839

# Measuring nditer vs for loop for feature generation

In [29]:
%%timeit

empty_point_wkt = Point().wkt  # "POINT EMPTY"
# create a numpy array of size len(ids_list) with empty_point_wkt
starting_points = np.full((1, 5821), empty_point_wkt, dtype=object)

qgis_fields_list = []

for wkt in np.nditer(starting_points, flags=['refs_ok']):
    feat = ["vlayer_fields"]
    feat.append("datetime_obj")  # Set its attributes
    # Create geometry from WKT string
    feat.append(wkt.item())
    qgis_fields_list.append(feat)
        

962 µs ± 39.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [31]:
%%timeit

empty_point_wkt = Point().wkt  # "POINT EMPTY"

qgis_fields_list = []

for i in range(5821):
    feat = ["vlayer_fields"]
    feat.append("datetime_obj")  # Set its attributes
    # Create geometry from WKT string
    feat.append(empty_point_wkt)
    qgis_fields_list.append(feat)

378 µs ± 2.94 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# Removing old files

In [62]:
import shutil

# Path to the directory
directory_path = f'/home/ali/matrices/'

shutil.rmtree(directory_path)


In [63]:
# create directory to store the matrices
os.makedirs(directory_path)